# Day 1-00.06: 모델 평가 개념 정리

이 노트북은 **목업 데이터**로 평가 지표 계산 흐름만 연습합니다. 실제 모델 응답을 넣어 평가하고 싶다면 main-practice/04 노트북을 참조하세요.

### 1. 필요한 라이브러리 불러오기

In [ ]:
import pandas as pd
import math
import re
from collections import Counter

print("✅ 라이브러리 로드 완료")

### 2. 평가에 사용할 예시 데이터
- 질문/정답(Reference)과 기준 모델/LoRA 모델 답변이 준비돼 있다고 가정합니다.
- 실제 모델 응답을 얻었다면 이 데이터 대신 동일한 구조로 교체하면 됩니다.

In [ ]:
sample_data = [
    {
        "question": "질문: 인공지능이란 무엇인가요?",
        "reference": "인공지능은 데이터를 학습해 패턴을 찾고 의사결정을 자동화하는 컴퓨터 기술입니다.",
        "baseline_answer": "인공지능은 사람이 만든 규칙을 실행하는 프로그램입니다.",
        "lora_answer": "인공지능은 데이터를 학습해 스스로 패턴을 찾아 의사결정을 돕는 기술입니다."
    },
    {
        "question": "질문: 머신러닝과 딥러닝의 차이점은?",
        "reference": "머신러닝은 사람이 특징을 정의한 뒤 학습시키고, 딥러닝은 신경망이 특징 추출까지 함께 수행합니다.",
        "baseline_answer": "머신러닝은 모델을 학습시키고 딥러닝은 데이터를 수집합니다.",
        "lora_answer": "머신러닝은 사람이 특징을 설계하고, 딥러닝은 신경망이 특징 추출까지 자동으로 수행합니다."
    },
    {
        "question": "질문: 자연어 처리는 무엇인가요?",
        "reference": "자연어 처리는 인간의 언어를 컴퓨터가 이해하고 생성하도록 만드는 기술입니다.",
        "baseline_answer": "자연어 처리는 언어 데이터를 저장하는 기술입니다.",
        "lora_answer": "자연어 처리는 인간의 언어를 컴퓨터가 이해하고 생성하도록 돕는 인공지능 기술입니다."
    }
]

pd.DataFrame(sample_data)

### 3. 평가 지표 계산 함수 정의
- **ROUGE-1**: 단어 중복 기반 리콜
- **BLEU**: n-그램 정밀도 기반 점수 (단순 1~4그램 구현)
- **코사인 유사도**: 단어 빈도를 벡터로 변환해 각도를 비교

In [ ]:
TOKEN_PATTERN = re.compile(r"[\w가-힣]+", re.UNICODE)

def tokenize(text):
    return TOKEN_PATTERN.findall(text.lower())


def compute_rouge1(reference, candidate):
    ref_tokens = tokenize(reference)
    cand_tokens = tokenize(candidate)
    ref_counts = Counter(ref_tokens)
    overlap = 0
    for token in cand_tokens:
        if ref_counts[token] > 0:
            overlap += 1
            ref_counts[token] -= 1
    return overlap / len(ref_tokens) if ref_tokens else 0.0


def compute_bleu(reference, candidate):
    ref_tokens = tokenize(reference)
    cand_tokens = tokenize(candidate)
    if not cand_tokens:
        return 0.0

    precisions = []
    for n in range(1, 5):
        ref_ngrams = Counter(tuple(ref_tokens[i:i+n]) for i in range(len(ref_tokens) - n + 1))
        cand_ngrams = Counter(tuple(cand_tokens[i:i+n]) for i in range(len(cand_tokens) - n + 1))
        if len(cand_ngrams) == 0:
            precisions.append(0.0)
            continue
        overlap = 0
        for ngram, count in cand_ngrams.items():
            overlap += min(count, ref_ngrams.get(ngram, 0))
        precisions.append(overlap / sum(cand_ngrams.values()))

    if any(p == 0 for p in precisions):
        geo_mean = 0.0
    else:
        geo_mean = math.exp(sum(math.log(p) for p in precisions) / 4)

    ref_len = len(ref_tokens)
    cand_len = len(cand_tokens)
    if cand_len == 0:
        bp = 0.0
    elif cand_len > ref_len:
        bp = 1.0
    else:
        bp = math.exp(1 - ref_len / cand_len)

    return bp * geo_mean


def compute_cosine_similarity(reference, candidate):
    ref_tokens = tokenize(reference)
    cand_tokens = tokenize(candidate)
    vocab = set(ref_tokens) | set(cand_tokens)
    if not vocab:
        return 0.0

    ref_counts = Counter(ref_tokens)
    cand_counts = Counter(cand_tokens)
    dot = sum(ref_counts[token] * cand_counts[token] for token in vocab)
    ref_norm = math.sqrt(sum(count * count for count in ref_counts.values()))
    cand_norm = math.sqrt(sum(count * count for count in cand_counts.values()))
    if ref_norm == 0 or cand_norm == 0:
        return 0.0

    return dot / (ref_norm * cand_norm)

print("✅ 평가 지표 함수 정의 완료")

### 4. 기준 모델과 LoRA 모델 점수 비교

In [ ]:
results = []

for row in sample_data:
    reference = row["reference"]
    baseline = row["baseline_answer"]
    lora = row["lora_answer"]

    results.append({
        "질문": row["question"],
        "기준 ROUGE-1": compute_rouge1(reference, baseline),
        "LoRA ROUGE-1": compute_rouge1(reference, lora),
        "기준 BLEU": compute_bleu(reference, baseline),
        "LoRA BLEU": compute_bleu(reference, lora),
        "기준 코사인": compute_cosine_similarity(reference, baseline),
        "LoRA 코사인": compute_cosine_similarity(reference, lora),
    })

results_df = pd.DataFrame(results).round(3)
display(results_df)

### 5. 평균 점수 비교

In [ ]:
metric_summary = pd.DataFrame({
    "Metric": ["ROUGE-1", "BLEU", "코사인 유사도"],
    "Baseline": [
        results_df["기준 ROUGE-1"].mean(),
        results_df["기준 BLEU"].mean(),
        results_df["기준 코사인"].mean(),
    ],
    "LoRA": [
        results_df["LoRA ROUGE-1"].mean(),
        results_df["LoRA BLEU"].mean(),
        results_df["LoRA 코사인"].mean(),
    ],
}).round(3)

display(metric_summary)

for metric, delta in zip(metric_summary["Metric"], metric_summary["LoRA"] - metric_summary["Baseline"]):
    arrow = "🔼" if delta > 0 else ("🔽" if delta < 0 else "➖")
    print(f"{arrow} {metric}: {delta:+.3f}")

### 6. 결과 해석 및 실제 적용 방법
- 예시에서는 LoRA 답변이 기준 모델보다 모든 지표에서 더 높도록 구성되어 있습니다.
- 실제 평가를 하려면 `sample_data` 대신 **모델이 생성한 응답**을 넣어 동일한 함수를 호출하면 됩니다.
- 추가 지표가 필요하면 METEOR, BERTScore 등도 같은 구조로 확장할 수 있습니다.

### 마무리
- 개념 이해가 끝났다면 main-practice/04 노트북에서 실제 LoRA 모델을 불러와 평가 지표를 계산해 보세요.
- 모델 로드 및 진짜 응답 생성은 main-practice/03 + 04, 이 노트북은 “지표 계산 흐름”을 익히는 용도입니다.